The goal of this notebook

In [ ]:
def create_parameter_names_dict():
    data = {
        key.upper(): value for key, value in {
            'FUN_fracfixers': 0, 'KCN': 1, 'a_fix': 2, 'crit_dayl': 3, 'd_max': 4, 'fff': 5,
            'froot_leaf': 6, 'fstor2tran': 7, 'grperc': 8, 'jmaxb0': 9, 'jmaxb1': 10, 'kcha': 11,
            'kmax': 12, 'krmax': 13, 'leaf_long': 14, 'leafcn': 15, 'lmr_intercept_atkin': 16,
            'lmrha': 17, 'lmrhd': 18, 'medlynintercept': 19, 'medlynslope': 20, 'nstem': 21,
            'psi50': 22, 'q10_mr': 23, 'slatop': 24, 'soilpsi_off': 25, 'stem_leaf': 26,
            'sucsat_sf': 27, 'theta_cj': 28, 'tpu25ratio': 29, 'tpuse_sf': 30, 'wc2wjb0': 31
        }.items()
    }
    return data


    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----      Split Data 90/10        ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # data for splitting
    X_train, X_test, y_train, y_test = train_test_split(param,
                                                        var,
                                                        test_size=0.2,
                                                        # setting a seed
                                                        random_state=0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----    Kernel Specs No Tuning    ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # initiate the model without tuning
    kernel = ConstantKernel(constant_value = 3,
                            constant_value_bounds=(1e-2, 1e4)) \
                  * RBF(length_scale=1, 
                        length_scale_bounds=(1e-4, 1e8))
   


 for param_index in range(32):
     # this needs to be X_train data for each parameters to iterate thru
            X_values = np.full((10, 32), 0.5)
            X_values[:, param_index] = np.linspace(0, 1, 10)
            y_pred, _ = gpr_model.predict(X_values, return_std=True)



 # using an out of the box kernel for now
    gpr_model = GaussianProcessRegressor(kernel=kernel,
                                        # want 20 random starts
                                        n_restarts_optimizer=20,
                                        # setting seed
                                        random_state=99,
                                        normalize_y=True)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Fit the Model        ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Fit the model to the training data
    gpr_model = gpr_model.fit(X_train, y_train)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Get Predictions      ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Make predictions
    y_pred, y_std = gpr_model.predict(X_test, return_std=True)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Collect Metrics      ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Calculate Mean Absolute Error
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate R^2
    r2_emulator = np.corrcoef(y_test, y_pred)[0,1]**2
    
    # Calculate RMSE
    rmse_emulator = np.sqrt(mean_squared_error(y_test, y_pred))
   
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----      Pickle Emulation     ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # save the model to disk
    pickle.dump(gpr_model, open(filename, 'wb')) 
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----        Print Metrics         ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Print Training Metrics
    print("R^2:", r2_emulator)
    print("RMSE:", rmse_emulator)
    print("Mean Absolute Error:", mae)


    sorted_indices = np.argsort(y_pred, y_std)
    sorted_parameter_emulations = np.array(y_pred, y_test)[sorted_indices]
    swapped_param_keys = {v: k for k, v in create_parameter_names_dict().items()}
    sorted_parameter_names = [swapped_param_keys[index] for index in sorted_indices]

    
    return gpr_model, y_pred, y_std, y_test, X_test, r2_emulator

In [ ]:
def train_emulator2(param, var):
    def create_parameter_names_dict():
        data = {
            key.upper(): value for key, value in {
                'FUN_fracfixers': 0, 'KCN': 1, 'a_fix': 2, 'crit_dayl': 3, 'd_max': 4, 'fff': 5,
                'froot_leaf': 6, 'fstor2tran': 7, 'grperc': 8, 'jmaxb0': 9, 'jmaxb1': 10, 'kcha': 11,
                'kmax': 12, 'krmax': 13, 'leaf_long': 14, 'leafcn': 15, 'lmr_intercept_atkin': 16,
                'lmrha': 17, 'lmrhd': 18, 'medlynintercept': 19, 'medlynslope': 20, 'nstem': 21,
                'psi50': 22, 'q10_mr': 23, 'slatop': 24, 'soilpsi_off': 25, 'stem_leaf': 26,
                'sucsat_sf': 27, 'theta_cj': 28, 'tpu25ratio': 29, 'tpuse_sf': 30, 'wc2wjb0': 31
            }.items()
        }
        return data

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(param, var, test_size=0.2, random_state=0)

    # Define the kernel
    kernel = ConstantKernel(constant_value=3, constant_value_bounds=(1e-2, 1e4)) \
              * RBF(length_scale=1, length_scale_bounds=(1e-4, 1e8))

    # Iterate through each parameter index
    for param_index in range(32):
        # Initialize the model
        gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20, random_state=99, normalize_y=True)
        
        # Select the parameter to perturb
        X_values[:, param_index] = X_train[:, param_index]  # Perturb the selected parameter
        
        # Fit the model to the training data
        gpr_model.fit(X_values, y_train)

        # Make predictions on the test data
        y_pred, y_std = gpr_model.predict(X_test, return_std=True)

        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2_emulator = np.corrcoef(y_test, y_pred)[0, 1] ** 2
        rmse_emulator = np.sqrt(mean_squared_error(y_test, y_pred))

        # Print metrics
        print(f"Parameter: {param_index}, R^2: {r2_emulator}, RMSE: {rmse_emulator}, MAE: {mae}")

        # Save the model to disk
        filename = f"gpr_model_param_{param_index}.pkl"
        pickle.dump(gpr_model, open(filename, 'wb'))
        
        return gpr_model, y_pred, y_std, y_test, X_test, r2_emulator

In [ ]:
def train_emulator2(param, var):
    def create_parameter_names_dict():
        data = {
            key.upper(): value for key, value in {
                'FUN_fracfixers': 0, 'KCN': 1, 'a_fix': 2, 'crit_dayl': 3, 'd_max': 4, 'fff': 5,
                'froot_leaf': 6, 'fstor2tran': 7, 'grperc': 8, 'jmaxb0': 9, 'jmaxb1': 10, 'kcha': 11,
                'kmax': 12, 'krmax': 13, 'leaf_long': 14, 'leafcn': 15, 'lmr_intercept_atkin': 16,
                'lmrha': 17, 'lmrhd': 18, 'medlynintercept': 19, 'medlynslope': 20, 'nstem': 21,
                'psi50': 22, 'q10_mr': 23, 'slatop': 24, 'soilpsi_off': 25, 'stem_leaf': 26,
                'sucsat_sf': 27, 'theta_cj': 28, 'tpu25ratio': 29, 'tpuse_sf': 30, 'wc2wjb0': 31
            }.items()
        }
        return data

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(param, var, test_size=0.2, random_state=0)

    # Define the kernel
    kernel = ConstantKernel(constant_value=3, constant_value_bounds=(1e-2, 1e4)) \
              * RBF(length_scale=1, length_scale_bounds=(1e-4, 1e8))

    # Iterate through each parameter index
    for param_index in range(32):
        # Initialize the model
        gpr_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20, random_state=99, normalize_y=True)
        
        # Perturb the selected parameter
        X_values = X_train[:, param_index]
        
        # Fit the model to the training data
        gpr_model.fit(X_values, y_train)

        # Make predictions on the test data
        y_pred, y_std = gpr_model.predict(X_test, return_std=True)

        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        r2_emulator = np.corrcoef(y_test, y_pred)[0, 1] ** 2
        rmse_emulator = np.sqrt(mean_squared_error(y_test, y_pred))

        # Print metrics
        print(f"Parameter: {param_index}, R^2: {r2_emulator}, RMSE: {rmse_emulator}, MAE: {mae}")

        # Save the model to disk
        filename = f"gpr_model_param_{param_index}.pkl"
        pickle.dump(gpr_model, open(filename, 'wb'))
        
    return gpr_model, y_pred, y_std, y_test, X_test, r2_emulator

In [ ]:
gpr_model, y_pred, y_std, y_test, X_test, r2_emulator = train_emulator2(param2, var2)